In [ ]:
from __future__ import division
import os
import sys
import subprocess as sb
import glob
import shutil
import multiprocessing

try:
    import cPickle as cp
except:
    import pickle as cp

import pandas as pd
import matplotlib as mpl

mpl.use('Agg')
import pylab as pl
import xml.etree.cElementTree as ET
from pybedtools import BedTool
from bioutilities import Genome_2bit
import pysam
import re
import numpy as np
from scipy.stats import zscore
import  urllib2    

In [10]:
HAYSTACK_VERSION = "0.4.0"

import logging

logging.basicConfig(level=logging.DEBUG,
                    format='%(levelname)-5s @ %(asctime)s:\n\t %(message)s \n',
                    datefmt='%a, %d %b %Y %H:%M:%S',
                    stream=sys.stderr,
                    filemode="w", filename='example.log'
                    )

error = logging.critical
warn = logging.warning
debug = logging.debug
info = logging.info
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("start")


In [11]:
#commmon functions haystack hotspots

def check_library(library_name):
    try:
        return __import__(library_name)
    except:
        error('You need to install %s module to use haystack!' % library_name)
        sys.exit(1)

def which(program):
    import os
    def is_exe(fpath):
        return os.path.isfile(fpath) and os.access(fpath, os.X_OK)

    fpath, fname = os.path.split(program)
    if fpath:
        if is_exe(program):
            return program
    else:
        for path in os.environ["PATH"].split(os.pathsep):
            path = path.strip('"')
            exe_file = os.path.join(path, program)
            if is_exe(exe_file):
                return exe_file
    return None

def check_program(binary_name, download_url=None):
    if not which(binary_name):
        error(
            'You need to install and have the command #####%s##### in your PATH variable to use CRISPResso!\n Please read the documentation!' % binary_name)
        if download_url:
            error('You can download it from here:%s' % download_url)
        sys.exit(1)

def check_file(filename):
    try:
        with open(filename):
            pass
    except IOError:
        raise Exception('I cannot open the file: ' + filename)


def quantile_normalization(A):
    AA = np.zeros_like(A)
    I = np.argsort(A, axis=0)
    AA[I, np.arange(A.shape[1])] = np.mean(A[I, np.arange(A.shape[1])], axis=1)[:, np.newaxis]

    return AA


def smooth(x, window_len=200):
    s = np.r_[x[window_len - 1:0:-1], x, x[-1:-window_len:-1]]
    w = np.hanning(window_len)
    y = np.convolve(w / w.sum(), s, mode='valid')
    return y[int(window_len / 2):-int(window_len / 2) + 1]


# write the IGV session file
def rem_base_path(path, base_path):
    return path.replace(os.path.join(base_path, ''), '')


def find_th_rpm(df_chip, th_rpm):
    return np.min(df_chip.apply(lambda x: np.percentile(x, th_rpm)))


def log2_transform(x):
    return np.log2(x + 1)


def angle_transform(x):
    return np.arcsin(np.sqrt(x) / 1000000.0)


def download_genome(name, output_directory=None):
    urlpath = "http://hgdownload.cse.ucsc.edu/goldenPath/%s/bigZips/%s.2bit" % (name, name)
    genome_url_origin = urllib2.urlopen(urlpath)

    genome_filename = os.path.join(output_directory, "%s.2bit" % name)

    print 'Downloading %s in %s...' % (urlpath, genome_filename)

    if os.path.exists(genome_filename):
        print 'File %s exists, skipping download' % genome_filename
    else:

        with open(genome_filename, 'wb') as genome_file_destination:
            shutil.copyfileobj(genome_url_origin, genome_file_destination)

        print 'Downloded %s in %s:' % (urlpath, genome_filename)

    g = Genome_2bit(genome_filename, verbose=True)

    chr_len_filename = os.path.join(output_directory, "%s_chr_lengths.txt" % name)
    if not os.path.exists(chr_len_filename):
        print 'Extracting chromosome lengths'
        g.write_chr_len(chr_len_filename)
        print 'Done!'
    else:
        print 'File %s exists, skipping generation' % chr_len_filename

    meme_bg_filename = os.path.join(output_directory, "%s_meme_bg" % name)
    if not os.path.exists(meme_bg_filename):
        print 'Calculating nucleotide frequencies....'
        g.write_meme_background(meme_bg_filename)
        print 'Done!'
    else:
        print 'File %s exists, skipping generation' % meme_bg_filename
        
        
def determine_path(folder):
    _ROOT=os.getcwd()
    return os.path.join(_ROOT,folder)          

In [ ]:
# added functions

def create_tiled_genome(genome_sorted_filtered_bins_file):

    chr_len_filtered_filename = os.path.join(genome_directory, "%s_chr_lengths_filtered.txt" % genome_name)

    genome_sorted_bins_file = os.path.join(output_directory,
                                           '%s.%dbp.bins.sorted.bed' % (os.path.basename(genome_name),
                                                                        bin_size))

    with open(chr_len_filtered_filename, 'wb') as f:
        f.writelines(line for line in open(chr_len_filename) if not re.search(chrom_exclude, line.split()[0]))

    genome_windows = BedTool().\
        window_maker(g=chr_len_filtered_filename,w=bin_size).\
        sort().\
        saveas(genome_sorted_bins_file)

    genome_windows.intersect(blacklist,
                             wa=True,
                             v=True,
                             output=genome_sorted_filtered_bins_file)

    try:
        os.remove(genome_sorted_bins_file)
    except:
        pass



def normalize_count(feature, scalar):
    feature.name = str(int(feature.name) * scalar)
    return feature


def get_scaling_factor(bam_filename):

    infile = pysam.AlignmentFile(bam_filename, "rb")
    numreads = infile.count(until_eof=True)
    scaling_factor = (1.0 / float(numreads)) * 1000000

    return scaling_factor

def filter_dedup_bam(bam_filename, bam_filtered_nodup_filename):


    bam_temp_filename=os.path.join(os.path.dirname(bam_filtered_nodup_filename),
                                       '%s.temp%s' % os.path.splitext(os.path.basename(bam_filtered_nodup_filename)))

    info('Removing  unmapped, mate unmapped, not primary alignment, low MAPQ reads, and reads failing qc')

    cmd = 'sambamba view -f bam -l 0 -t %d -F "not (unmapped or mate_is_unmapped or failed_quality_control or duplicate or secondary_alignment) and mapping_quality >= 30" "%s"  -o "%s"' % (
        n_processes, bam_filename, bam_temp_filename)
    proc = sb.call(cmd, shell=True)

    info('Removing  optical duplicates')

    cmd = 'sambamba markdup  -l 5 -t %d --hash-table-size=17592186044416 --overflow-list-size=20000000 --io-buffer-size=256 "%s" "%s" ' % (
        n_processes, bam_temp_filename, bam_filtered_nodup_filename)
    proc = sb.call(cmd, shell=True)

    try:
        os.remove(bam_temp_filename)
        os.remove(bam_temp_filename + '.bai')
    except:
        pass

def to_bedgraph(bam_filtered_nodup_filename, ext, rpm_filename):

    info('Converting bam to bed and extending read length...')
    bed_extended= BedTool(bam_filtered_nodup_filename). \
        bam_to_bed(). \
        slop(r=ext, l=0, s=True, g=chr_len_filename)

    info('Computing Scaling Factor...')
    scaling_factor = get_scaling_factor(bam_filtered_nodup_filename)

    info('Computing coverage over bins...')
    info('Normalizing counts by scaling factor...')

    bedgraph = BedTool(genome_sorted_filtered_bins_file). \
        intersect(bed_extended, c=True). \
        each(normalize_count, scaling_factor).saveas(rpm_filename)

    info('Bedgraph saved...')

    return bedgraph

def to_bigwig(bigwig_filename, rpm_filename):

    if which('bedGraphToBigWig'):
        if not os.path.exists(bigwig_filename) or recompute_all:
            cmd = 'bedGraphToBigWig "%s" "%s" "%s"' % (rpm_filename, chr_len_filename, bigwig_filename)
            proc = sb.call(cmd, shell=True)

            try:
                os.remove(rpm_filename)

            except:
                pass

    else:
        info(
            'Sorry I cannot create the bigwig file.\nPlease download and install bedGraphToBigWig from here: http://hgdownload.cse.ucsc.edu/admin/exe/ and add to your PATH')

def average_bigwig(bam_filename, binned_rpm_filename,bigwig_filename):

    cmd = 'bigWigAverageOverBed %s %s  /dev/stdout | sort -s -n -k 1,1 | cut -f5 > %s' % (
        bam_filename, genome_sorted_filtered_bins_file, binned_rpm_filename)
    sb.call(cmd, shell=True)
    shutil.copy2(bam_filename, bigwig_filename)


In [ ]:
samples_filename= determine_path('test_data/samples_names.txt')
output_directory=determine_path( 'haystack_analysis/output')
motif_directory= determine_path('motif_daatabases')
annotation_directory=determine_path('gene_annotations')
genome_directory=determine_path('genomes')

NameError: global name 'os' is not defined

In [19]:
'''
#mandatory
parser = argparse.ArgumentParser(description='HAYSTACK Parameters')
parser.add_argument('samples_filename_or_bam_folder', type=str,  help='A tab delimeted file with in each row (1) a sample name, (2) the path to the corresponding bam filename, (3 optional) the path to the corresponding gene expression filaneme. Alternatively it is possible to specify a folder containing some .bam files to analyze.')
parser.add_argument('genome_name', type=str,  help='Genome assembly to use from UCSC (for example hg19, mm9, etc.)')

#optional
parser.add_argument('--name',  help='Define a custom output filename for the report', default='')
parser.add_argument('--output_directory',type=str, help='Output directory (default: current directory)',default='')
parser.add_argument('--bin_size', type=int,help='bin size to use(default: 500bp)',default=500)
parser.add_argument('--recompute_all',help='Ignore any file previously precalculated fot the command haystack_hotstpot',action='store_true')
parser.add_argument('--depleted', help='Look for cell type specific regions with depletion of signal instead of enrichment',action='store_true')
parser.add_argument('--input_is_bigwig', help='Use the bigwig format instead of the bam format for the input. Note: The files must have extension .bw',action='store_true')
parser.add_argument('--disable_quantile_normalization',help='Disable quantile normalization (default: False)',action='store_true')
parser.add_argument('--transformation',type=str,help='Variance stabilizing transformation among: none, log2, angle (default: angle)',default='angle',choices=['angle', 'log2', 'none'])
parser.add_argument('--z_score_high', type=float,help='z-score value to select the specific regions(default: 1.5)',default=1.5)
parser.add_argument('--z_score_low', type=float,help='z-score value to select the not specific regions(default: 0.25)',default=0.25)
parser.add_argument('--th_rpm',type=float,help='Percentile on the signal intensity to consider for the hotspots (default: 99)', default=99)
parser.add_argument('--meme_motifs_filename', type=str, help='Motifs database in MEME format (default JASPAR CORE 2016)')
parser.add_argument('--motif_mapping_filename', type=str, help='Custom motif to gene mapping file (the default is for JASPAR CORE 2016 database)')
parser.add_argument('--plot_all',  help='Disable the filter on the TF activity and correlation (default z-score TF>0 and rho>0.3)',action='store_true')
parser.add_argument('--n_processes',type=int, help='Specify the number of processes to use. The default is #cores available.',default=multiprocessing.cpu_count())
parser.add_argument('--temp_directory',  help='Directory to store temporary files  (default: /tmp)', default='/tmp')
parser.add_argument('--version',help='Print version and exit.',action='version', version='Version %s' % HAYSTACK_VERSION)
args = parser.parse_args()
'''

# HAYSTACK Parameters

samples_filename_or_bam_folder = samples_filename
genome_name = 'hg19'

# optional
name = ''
output_directory = output_directory
bin_size = 200
recompute_all = True
depleted = True
input_is_bigwig = False
disable_quantile_normalization = False
transformation = 'angle'
z_score_high = 1.5
z_score_low = 0.25
th_rpm = 99
meme_motifs_filename = '' #os.path.join(MOTIF_DIR, 'JASPAR_CORE_2016_vertebrates.meme')
motif_mapping_filename = '' #os.path.join(MOTIF_DIR, 'JASPAR_CORE_2016_vertebrates_mapped_to_gene_human_mouse.txt')
plot_all = True
use_X_Y= True
chrom_exclude = '' #'_|chrM|chrX|chrY'
ext=200
n_processes = multiprocessing.cpu_count()
temp_directory = '' #os.path.join(_ROOT, 'tmp')
version = 'Version %s' % HAYSTACK_VERSION

In [20]:
if meme_motifs_filename:
    check_file(meme_motifs_filename)

if motif_mapping_filename:
    check_file(motif_mapping_filename)

# if not os.path.exists(temp_directory):
#    error('The folder specified with --temp_directory: %s does not exist!' % temp_directory)
#    sys.exit(1)

if input_is_bigwig:
    extension_to_check = '.bw'
    info('Input is set BigWig (.bw)')
else:
    extension_to_check = '.bam'
    info('Input is set compressed SAM (.bam)')

if name:
    directory_name = 'HAYSTACK_PIPELINE_RESULTS_on_%s' % name

else:
    directory_name = 'HAYSTACK_PIPELINE_RESULTS'

if output_directory:
    output_directory = os.path.join(output_directory, directory_name)
else:
    output_directory = directory_name

# check folder or sample filename

USE_GENE_EXPRESSION = True

if os.path.isfile(samples_filename_or_bam_folder):
    BAM_FOLDER = False
    bam_filenames = []
    gene_expression_filenames = []
    sample_names = []

    dir_path = os.path.dirname(os.path.realpath(samples_filename_or_bam_folder))

    with open(samples_filename_or_bam_folder) as infile:
        for line in infile:

            if not line.strip():
                continue

            if line.startswith('#'):  # skip optional header line or empty lines
                info('Skipping header/comment line:%s' % line)
                continue

            fields = line.strip().split("\t")
            n_fields = len(fields)

            if n_fields == 2:

                USE_GENE_EXPRESSION = False

                sample_names.append(fields[0])
                bam_filenames.append(fields[1])

            elif n_fields == 3:

                USE_GENE_EXPRESSION = USE_GENE_EXPRESSION and True

                sample_names.append(fields[0])
                bam_filenames.append(fields[1])
                gene_expression_filenames.append(fields[2])
            else:
                error('The samples file format is wrong!')

    bam_filenames = [os.path.join(dir_path, filename) for filename in bam_filenames]
    gene_expression_filenames = [os.path.join(dir_path, filename) for filename in gene_expression_filenames]

else:
    if os.path.exists(samples_filename_or_bam_folder):
        BAM_FOLDER = True
        USE_GENE_EXPRESSION = False
        bam_filenames = glob.glob(os.path.join(samples_filename_or_bam_folder, '*' + extension_to_check))

        if not bam_filenames:
            error('No bam/bigwig  files to analyze in %s. Exiting.' % samples_filename_or_bam_folder)
            sys.exit(1)

        sample_names = [os.path.basename(bam_filename).replace(extension_to_check, '') for bam_filename in
                        bam_filenames]
    else:
        error("The file or folder %s doesn't exist. Exiting." % samples_filename_or_bam_folder)
        sys.exit(1)

# check all the files before starting
info('Checking samples files location...')
for bam_filename in bam_filenames:
    check_file(bam_filename)

if USE_GENE_EXPRESSION:
    for gene_expression_filename in gene_expression_filenames:
        check_file(gene_expression_filename)

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# copy back the file used
if not BAM_FOLDER:
    shutil.copy2(samples_filename_or_bam_folder, output_directory)

# write hotspots conf files
sample_names_hotspots_filename = os.path.join(output_directory, 'sample_names_hotspots.txt')

with open(sample_names_hotspots_filename, 'w+') as outfile:
    for sample_name, bam_filename in zip(sample_names, bam_filenames):
        outfile.write('%s\t%s\n' % (sample_name, bam_filename))

# write tf activity  conf files
if USE_GENE_EXPRESSION:
    sample_names_tf_activity_filename = os.path.join(output_directory, 'sample_names_tf_activity.txt')

    with open(sample_names_tf_activity_filename, 'w+') as outfile:
        for sample_name, gene_expression_filename in zip(sample_names, gene_expression_filenames):
            outfile.write('%s\t%s\n' % (sample_name, gene_expression_filename))

    tf_activity_directory = os.path.join(output_directory, 'HAYSTACK_TFs_ACTIVITY_PLANES')


if name:
    directory_name = 'HAYSTACK_HOTSPOTS_on_%s' % name

else:
    directory_name = 'HAYSTACK_HOTSPOTS'

if output_directory:
    output_directory = os.path.join(output_directory, directory_name)
else:
    output_directory = directory_name

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

tracks_directory = os.path.join(output_directory, 'TRACKS')
if not os.path.exists(tracks_directory):
    os.makedirs(tracks_directory)

intermediate_directory = os.path.join(output_directory, 'INTERMEDIATE')
if not os.path.exists(intermediate_directory):
    os.makedirs(intermediate_directory)
    

In [ ]:

info('Initializing Genome:%s' % genome_name)

genome_2bit = os.path.join(genome_directory, genome_name + '.2bit')

if os.path.exists(genome_2bit):
    genome = Genome_2bit(genome_2bit)
else:
    info("\nIt seems you don't have the required genome file.")

    download_genome(genome_name, genome_directory)
    if os.path.exists(genome_2bit):
        info('Genome correctly downloaded!')
        genome = Genome_2bit(genome_2bit)
    else:
        error('Sorry I cannot download the required file for you. Check your Internet connection.')
        sys.exit(1)


In [ ]:

chr_len_filename = os.path.join(genome_directory, "%s_chr_lengths.txt" % genome_name)
blacklist = os.path.join(genome_directory,'blacklist.bed')
check_file(chr_len_filename)
check_file(blacklist)

genome_sorted_filtered_bins_file = os.path.join(output_directory,
                                                    '%s.%dbp.bins.sorted.filterd.bed' % (os.path.basename(genome_name),
                                                                                         bin_size))

if not os.path.exists(genome_sorted_filtered_bins_file) or recompute_all:
    info('Creating bins of %dbp in %s' % (bin_size, genome_sorted_filtered_bins_file))
    create_tiled_genome(genome_sorted_filtered_bins_file)


In [ ]:
# convert bam files to genome-wide rpm tracks
for base_name, bam_filename in zip(sample_names, bam_filenames):

    # bam_filename=bam_filenames[0]
    # base_name=sample_names[0]

    info('Processing:%s' % bam_filename)

    rpm_filename = os.path.join(tracks_directory, '%s.bedgraph' % base_name)
    binned_rpm_filename = os.path.join(intermediate_directory, '%s.%dbp.rpm' % (base_name, bin_size))
    bigwig_filename = os.path.join(tracks_directory, '%s.bw' % base_name)
    bam_filtered_nodup_filename = os.path.join(dir_path,
                                               '%s.filtered.nodup%s' % (os.path.splitext(
                                                   os.path.basename(bam_filename))))

    if not os.path.exists(binned_rpm_filename) or recompute_all:

        if input_is_bigwig and which('bigWigAverageOverBed'):

            average_bigwig(bam_filename, binned_rpm_filename, bigwig_filename)

        else:
            info('Filtering and deduping BAM file...')

            filter_dedup_bam(bam_filename, bam_filtered_nodup_filename)

            info('Building BedGraph RPM track...')

            bedgraph = to_bedgraph(bam_filtered_nodup_filename, ext, rpm_filename)

            info('Converting BedGraph to BigWig')

            to_bigwig(bigwig_filename, rpm_filename)

            info('Making constant binned (%dbp) rpm values file' % bin_size)

            bedgraph.to_dataframe()['name'].to_csv(binned_rpm_filename)